In [66]:
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict, Counter
from ebaysdk.shopping import Connection as Shopping
shopping = Shopping(appid='GabrielO-Flipper-PRD-6246ab075-45501e29', config_file='ebay.yaml')
import json
import os
import copy
import re
date_regex = re.compile(r"^(SOLD.+\d{4})")
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
x = json.loads(open("test_data.json").read())
q = {}
items = {}
items_by_search = defaultdict(dict)
titles_by_search = defaultdict(list)
titles = []
long_titles = []
data_exclude = ["title","shipping"]
error_listings = []
id_counter = Counter()
item_count = 0
found_count = 0
for k in list(x.keys()):
    try:
        y = x[k]["listings"]
    except:
        print(k)
        continue
    search = k.split("_pg")[0]
    data_keys = x[k]["metadata"]["data_keys"]["listings"]["children"]
    found_count += len(y.keys())
    for i in y.keys():
        
        children = y[i]["children"][0]
        title = ""
        link = ""
        price = 0
        measurements = {}
        quantity = {}
        pid = []
        data_from_fields = {}
        all_fields = list(data_keys.keys())
        child_keys = [x.rpartition("_")[0] for x in children.keys()]
        for c in children.keys():
            
            key_split = c.rpartition("_")[0]
            try:
                all_fields.remove(key_split)
            except:
                pass
            if key_split in data_exclude:
                if "title" in c:
                    title = children[c]["title"].strip()
                    
                    title = title.replace("\n","").replace("\t","")
                    if "new listing" in title.lower():
                        title = " ".join(title.split()[2:])
                        
                    tmp_measure = {}
                    for m in children[c]["measurements"].keys():
                        tmp_measure[m] = list(set(children[c]["measurements"][m]))
                    measurements =tmp_measure
                    title_short = children[c]["title_shortened"]
                    quan_tmp = defaultdict(list)
                    for quan in children[c]["quantity"].keys():
                        quan_tmp[quan].append(children[c]["quantity"][quan])
                            
                    quantity = dict(quan_tmp)
                    pid = children[c]["product_ids"]
                    link = children[c]["link"]
                    if "sold_on" not in child_keys and children[c]["sold_on"]:
                        sold_on = children[c]["sold_on"]
                    elif "sold_on" not in child_keys:
                        #print(i)
                        pass
                    else:
                        sold_on = ""
                        
                elif "shipping" in c:
                    ship_info = children[c]["shipping"]
                    
            else:
                key_fields = data_keys[key_split]
                if key_split == "price" and not children[c][key_split]:
                    print(k)
                    print(children["title_1"]["title"])
                if key_fields["text"]:
                    data_from_fields[key_split] = children[c][key_split]
                for field in key_fields["keys"]:
                    data_from_fields[field] = children[c][field]
                    
            
        e_id = link.split(r"/")[-1].split("?")[0]
        if title != "ELEM_MEMBER_TEXT_NONE" and title != "new listing":
            item_count+=1
            long_titles.append(title)
            titles.append(title_short)
            tmp_d = {"quantity":quantity,"measurements":measurements,"title_short":title_short,"title":title,"eid":e_id,"pid":pid,"link":link}
            tmp_d = {**tmp_d,**data_from_fields}
            for field in all_fields:
                if field not in tmp_d:
                    tmp_d[field] = "None Found"
            
            tmp_d["i_format"] = copy.deepcopy(tmp_d["format"])
            del tmp_d["format"]
            
            
            
            id_counter["item_"+str(e_id)] += 1
            if "item_"+str(e_id) in items_by_search[search] or "item_"+str(e_id) in items :
                key_use = "item_"+str(e_id)+"_"+str(id_counter["item_"+str(e_id)])
            else:
                key_use = "item_"+str(e_id)
                
            if "item_150626601803" in key_use:
                print(k)
                print(i)
                print(tmp_d["sold_on"])
            items_by_search[search][key_use] =  tmp_d
            items[key_use] = tmp_d
            titles_by_search[search].append(title)
        else:
            error_listings.append(y[i])

In [40]:
x["generic_industrial_capacitors_completed_pg36"]["listings"]["result_20"]

{'children': [{'condition_1': {'children': [],
    'condition': 'Brand New',
    'num_children': 0},
   'format_1': {'children': [], 'format': 'Buy It Now', 'num_children': 0},
   'location_1': {'children': [], 'location': 'From China', 'num_children': 0},
   'price_1': {'children': [], 'num_children': 0, 'price': [3.68]},
   'shipping_1': {'children': [],
    'num_children': 0,
    'shipping': '+$2.00 shipping'},
   'sold_on_1': {'children': [], 'num_children': 0, 'sold_on': 'Mar-04 18:17'},
   'title_1': {'children': [],
    'dimensions': [],
    'link': 'https://www.ebay.com/itm/25V-600UF-650UF-1000UF-2000UF-Non-polar-axial-electrolytic-capacitor-DIY-audio/132518742150?hash=item1edabb8886:m:mIgDMrfPoTEiTeOTGz6no9g',
    'measurements': {'microfarads': [600.0, 650.0, 1000.0, 2000.0],
     'volts': [25.0]},
    'num_children': 0,
    'product_ids': [],
    'quantity': {'all': 1},
    'sold_on': '',
    'title': '25V 600UF/650UF/1000UF/2000UF Non-polar axial electrolytic capacitor DIY 

In [19]:
items_by_search["generic_industrial_capacitors_completed"]["item_173087366333"]


{'condition': 'Brand New',
 'eid': '173087366333',
 'exact_sold_on': 'None Found',
 'i_format': 'Buy It Now',
 'link': 'https://www.ebay.com/itm/CBB60-AC-450V-6-10-12-16-20uF-Wired-Motor-Run-Start-SH-Capacitor-50-60Hz-Ka/173087366333?hash=item284ccf9cbd:m:mgLkX2jlIECQMFAk-qkCJsw',
 'location': 'None Found',
 'measurements': {'hertz': [60.0], 'microfarads': [20.0], 'volts': [450.0]},
 'pid': ['cbb60'],
 'price': [9.1],
 'quantity': {'all': [1]},
 'sold_on': 'Feb-27 23:26',
 'title': 'CBB60 AC 450V 6/10/12/16/20uF Wired Motor Run Start SH Capacitor 50/60Hz Ka',
 'title_short': 'cbb60 ac 6/10/12/16/ wired motor run start sh capacitor 50/ ka'}

In [52]:
import datetime
current_date = datetime.date.today().strftime("%B %d %Y")
path_to_date = os.path.join(os.getcwd()+r"\data", current_date)
if not os.path.isdir(path_to_date):
    os.mkdir(path_to_date)
items_by_search = dict(items_by_search)
current_time = datetime.datetime.now().strftime("%I%p")
for i in items_by_search.keys():
    with open(path_to_date+"\\"+i+".json","w", encoding='utf-8') as f:
        f.write(json.dumps(items_by_search[i], indent=4, sort_keys=True))
    with open(os.getcwd() + r"\parsed_data"+"\\"+i+".json","w", encoding='utf-8') as f:
        f.write(json.dumps(items, indent=4, sort_keys=True))
    with open(os.getcwd() + r"\parsed_data"+"\\"+i+".txt","w", encoding='utf-8') as f:
        for t in titles_by_search[i]:
            f.write(t.strip()+"\n")

In [10]:
qq = "asdasddfasd"
qq[1:]

'sdasddfasd'

In [30]:
items["item_152783443424"]

{'condition': 'Brand New',
 'eid': '152783443424',
 'exact_sold_on': 'None Found',
 'i_format': 'Buy It Now',
 'link': 'https://www.ebay.com/itm/Packard-PTMJ56-Motor-Start-Capacitor-53-64-MFD-220-250-VAC/152783443424?epid=2255980968&hash=item23929a55e0:g:WNoAAMXQfFJRU15i',
 'location': '',
 'measurements': {'millifarads': [64.0, 53.0], 'volts': [250.0, 220.0]},
 'pid': ['ptmj56'],
 'price': [6.75],
 'quantity': {'all': [1]},
 'sold_on': 'May-28 16:13',
 'title': 'Packard PTMJ56 Motor Start Capacitor 53-64 MFD 220-250 VAC',
 'title_short': 'packard ptmj56 motor start capacitor'}

In [67]:
id_counter.most_common()

[('item_391911455525', 3),
 ('item_162761215463', 2),
 ('item_372249400808', 2),
 ('item_401446551391', 2),
 ('item_192494356552', 2),
 ('item_232638429740', 2),
 ('item_252548081002', 2),
 ('item_282975044810', 2),
 ('item_282956436636', 2),
 ('item_272868012646', 2),
 ('item_192551370986', 2),
 ('item_132616165744', 2),
 ('item_162777760266', 2),
 ('item_232158557520', 2),
 ('item_371143923916', 2),
 ('item_262768733060', 2),
 ('item_110946655519', 2),
 ('item_232290694286', 2),
 ('item_332200827798', 2),
 ('item_261000080458', 2),
 ('item_222140140739', 2),
 ('item_122909727940', 2),
 ('item_192164220202', 2),
 ('item_322664976441', 2),
 ('item_382423580615', 2),
 ('item_201808049852', 2),
 ('item_202298614259', 2),
 ('item_272226479574', 2),
 ('item_123049858040', 2),
 ('item_173002937184', 2),
 ('item_192453928465', 2),
 ('item_282636131200', 2),
 ('item_173014403570', 2),
 ('item_152199359150', 2),
 ('item_380334983525', 2),
 ('item_263178345836', 2),
 ('item_271776052607', 2),
 